In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from openfisca_france import CountryTaxBenefitSystem
from openfisca_france.model.base import Famille, FoyerFiscal, Menage
from openfisca_core import periods, parameters
from openfisca_core.model_api import Reform

In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os

In [ ]:
import sys
sys.path.append('../technique')

In [ ]:
from utils import *

In [ ]:
base_period = "2023-01"

base = CountryTaxBenefitSystem()
base.load_extension('openfisca_france_local')

In [ ]:
full_df = pd.read_excel(os.getenv('DATA_FOLDER') + "mobilite/extrait-Tableau_de_Bord_CTS_Valeurs 012023_ajout_qf_age.xlsx", sheet_name="QRD - Quantités")

In [ ]:
full_df = pd.read_excel(os.getenv('DATA_FOLDER') + "mobilite/extrait-Tableau_de_Bord_CTS_Valeurs 012023_ajout_qf_age.xlsx", sheet_name="QRD - Quantités")
full_df['ajustement_mensuel_num'] = full_df.ajustement_mensuel.astype('str').apply(lambda x: eval(str(x)) if x != "nan" else 1)

In [ ]:
compens_df = full_df[~full_df.Exclu.isna() * ~full_df.ajustement_mensuel.isna()][['Titres', 'quantité', 'ajustement_mensuel_num']]

In [ ]:
compens_constant = sum(compens_df.quantité * compens_df.ajustement_mensuel_num)
compens_constant

In [ ]:
df = full_df[full_df.Exclu.isna()]

In [ ]:
count = len(df)

individu_df = pd.DataFrame({
    'famille_id': list(range(count)),
    'qfrule': df.QF,
    'agerule': df.AGE,
    'taux_incapacite': np.where(df.Titres.str.contains('PMR'), 0.8, 0)
})

In [ ]:
determine_qf(individu_df, qfrules_constant)

In [ ]:
determine_age(individu_df)

In [ ]:
famille_df = pd.DataFrame({})
menage_df = pd.DataFrame({
    'eurometropole_strasbourg_tarification_solidaire_transport_eligibilite_geographique': np.ones(count),
})
foyerfiscaux_df = pd.DataFrame({})

individu_df['famille_role_index'] = 0
individu_df['foyer_fiscal_id'] = individu_df.famille_id
individu_df['foyer_fiscal_role_index'] = 0
individu_df['menage_id'] = individu_df.famille_id
individu_df['menage_role_index'] = 0

data = dict(input_data_frame_by_entity = dict(
individu=individu_df,
famille=famille_df,
menage=menage_df,
foyer_fiscal=foyerfiscaux_df))

In [ ]:
scenario = StrasbourgSurveyScenario(tbs=base, data = data)

In [ ]:
res = scenario.simulation.calculate('eurometropole_strasbourg_tarification_transport', base_period)

In [ ]:
res_elig_reduit = scenario.simulation.calculate('eurometropole_strasbourg_tarification_solidaire_transport_eligible_tarif_reduit', base_period)

In [ ]:
res_df = pd.DataFrame(data= {
    'idx': df.idx,
    'titre_fichier': df.Titres,
    'AGE': df.AGE,
    'QF': df.QF,
    'recettes_fichier': df.recettes,
    'recettes_calc_ht': res * df.quantité/1.1* df.ajustement_mensuel_num,
    'quantité_fichier': df.quantité,
    'pu_fichier': df.PU,
    'pu_calc': res,
    'pu_calc_ht': res/1.1* df.ajustement_mensuel_num,
    'reduit': res_elig_reduit,
})

res_df['ok'] = (res_df.pu_calc_ht - res_df.pu_fichier).abs()< 1
assert((~res_df['ok']).sum() == 3)
#res_df.to_excel('/home/thomas/Nextcloud/CodeursEnLiberte/EMSb/mobilite/recalcul_prix_unitaire_titres_2.xlsx')
# pd.pivot(res_df, columns=["QF"], index=["AGE"], values="quantité_fichier")

In [ ]:
def add_compensation(df):
    max_prix = df[['reduit', 'pu_calc']].groupby(by='reduit').max().pu_calc
    max_prix_grille = np.where(df.reduit, max_prix[1], max_prix[0])
    df['tp'] = df.pu_calc == max_prix_grille
    df['compensation'] = np.where(df['tp'], (max_prix_grille/max(max_prix_grille)).round(1), 0)

In [ ]:
add_compensation(res_df)

In [ ]:
pd.pivot_table(res_df, index=["tp"], values="quantité_fichier", aggfunc=sum)

In [ ]:
recette_base = res_df.recettes_calc_ht.sum()
recette_base

In [ ]:
count = int(sum(df.quantité))
sample_count = 2
(count, sample_count)

In [ ]:
sample_ids = np.repeat(list(range(sample_count)), count)
indiv_ids = np.tile(list(range(count)), sample_count)
sample_qfrule = np.tile(np.repeat(df.QF, df.quantité), sample_count)
ajustement_mensuel_num = np.tile(np.repeat(df.ajustement_mensuel_num, df.quantité), sample_count)

sample_individu_df = pd.DataFrame({
    'sample_id': sample_ids,
    'famille_id': list(range(count * sample_count)),
    'qfrule': sample_qfrule,
    'agerule': np.tile(np.repeat(df.AGE, df.quantité), sample_count),
    'taux_incapacite': np.tile(np.repeat(np.where(df.Titres.str.contains('PMR'), 0.8, 0), df.quantité), sample_count)
})

In [ ]:
def alea_caf_fiscal(a):
    return np.maximum(0, np.random.normal(a*0.9 - 150, np.maximum(0, (a*0.9 - 100)*0+100))).astype('int64')

In [ ]:
determine_qf(sample_individu_df, unif_qf, alea_caf_fiscal)

In [ ]:
determine_age(sample_individu_df)

In [ ]:
len(sample_individu_df)

In [ ]:
def build_data(sample_individu_df):
    sample_famille_df = pd.DataFrame({})
    sample_menage_df = pd.DataFrame({
        'eurometropole_strasbourg_tarification_solidaire_transport_eligibilite_geographique': np.ones(len(sample_individu_df)),
    })
    sample_foyerfiscaux_df = pd.DataFrame({})

    sample_individu_df['famille_role_index'] = 0
    sample_individu_df['foyer_fiscal_id'] = sample_individu_df.famille_id
    sample_individu_df['foyer_fiscal_role_index'] = 0
    sample_individu_df['menage_id'] = sample_individu_df.famille_id
    sample_individu_df['menage_role_index'] = 0

    sample_data = dict(input_data_frame_by_entity = dict(
    individu=sample_individu_df,
    famille=sample_famille_df,
    menage=sample_menage_df,
    foyer_fiscal=sample_foyerfiscaux_df))
    return sample_data

In [ ]:
sample_scenario = StrasbourgSurveyScenario(base, data = build_data(sample_individu_df))

In [ ]:
def compute_result(scenario):
    sample_calc = scenario.simulation.calculate('eurometropole_strasbourg_tarification_transport', base_period)
    sample_reduit = scenario.simulation.calculate('eurometropole_strasbourg_tarification_solidaire_transport_eligible_tarif_reduit', base_period)

    sample_res = pd.DataFrame(data= {
        "sample_id": sample_ids,
        "individu_id": indiv_ids,
        "recettes": sample_calc / 1.1 * ajustement_mensuel_num,
        "pu_calc": sample_calc,
        "pu_calc_ht": sample_calc / 1.1 * ajustement_mensuel_num,
        "reduit": sample_reduit,
        "idx": np.tile(np.repeat(df.idx, df.quantité), sample_count),
        "pu_fichier": np.tile(np.repeat(res_df.pu_fichier, df.quantité), sample_count),
        "pu_calc_base": np.tile(np.repeat(res_df.pu_calc, df.quantité), sample_count),
        "tp_base": np.tile(np.repeat(res_df.tp, df.quantité), sample_count),
    })
    sample_res['ecart'] = sample_res.pu_calc_ht - sample_res.pu_fichier
    add_compensation(sample_res)
    
    sample_recette = sample_res[['sample_id', "recettes"]].groupby(by="sample_id").sum()
    sample_recette['equiv_tp'] = sample_res[['sample_id', "compensation"]].groupby(by="sample_id").sum()
    sample_recette['tp'] = sample_res[['sample_id', "tp"]].groupby(by="sample_id").sum()
    sample_recette["pertes"] = sample_recette.recettes-recette_base
    sample_recette['compens'] = -sample_recette.pertes/(sample_recette.equiv_tp + compens_constant)

    return (sample_res, sample_recette)

In [ ]:
(sample_res, sample_recette) = compute_result(sample_scenario)

In [ ]:
sample_res

In [ ]:
pd.pivot_table(sample_res[["pu_calc_base", "pu_calc"]], index="pu_calc_base", columns="pu_calc", aggfunc=len, fill_value=0)

In [ ]:
sample_recette

In [ ]:
#res_df.idx[~res_df.ok]

In [ ]:
#sample_res[['idx', 'ecart']].groupby(by="idx").describe()

In [ ]:
sample_recette.describe()

In [ ]:
tp_df = pd.DataFrame(data= {
    "sample_id": sample_res.sample_id,
    "avant": sample_res.tp_base,
    "apres": sample_res.tp,
})

In [ ]:
pd.pivot_table(tp_df, index=["sample_id", "avant"], columns="apres", aggfunc=len)

In [ ]:
#print(
pd.pivot_table(res_df, index=["QF"], columns=["reduit"], values="quantité_fichier", aggfunc=sum)
#.to_csv(sep=";",decimal=","))

In [ ]:
if False:
    denomb_df = pd.DataFrame(data={
        'QF': res_df.QF,
        'plein': ~res_df.reduit,
        'quantité_fichier': res_df.quantité_fichier
    })
    denomb = pd.pivot_table(denomb_df, index=["QF", "plein"], values="quantité_fichier", aggfunc=sum)
    denomb
    merge_bareme_population = denomb.quantité_fichier.cumsum()
    df[["QF", "quantité"]].groupby(by="QF").sum().cumsum()

In [ ]:
if False:
    static_sample_count = 50
    static_sample_ids = np.repeat(list(range(static_sample_count)), count)
    static_sample_individu_df = pd.DataFrame({
        'sample_id': static_sample_ids,
        'famille_id': list(range(count * static_sample_count)),
        'qfrule': np.tile(np.repeat(df.QF, df.quantité), static_sample_count),
        'agerule': np.tile(np.repeat(df.AGE, df.quantité), static_sample_count),
        'taux_incapacite': np.tile(np.repeat(np.where(df.Titres.str.contains('PMR'), 0.8, 0), df.quantité), static_sample_count)
    })


    determine_qf(static_sample_individu_df,unif_qf, alea_caf_fiscal)

    rr = static_sample_individu_df.groupby(by="sample_id").eurometropole_strasbourg_tarification_solidaire_transport_quotient_familial.rank(method="first")

    df[["QF", "quantité"]].groupby(by="QF").sum()

    indexes = rr.isin(df[["QF", "quantité"]].groupby(by="QF").sum().cumsum().quantité)
    rr_res = pd.DataFrame(data={
        "rank": rr[indexes],
        "value": static_sample_individu_df.eurometropole_strasbourg_tarification_solidaire_transport_quotient_familial[indexes]
    })

    rr_res.groupby(by="rank").describe()

    no_reduit_indexes = rr.isin(merge_bareme_population)
    no_reduit_rr_res = pd.DataFrame(data={
        "rank": rr[no_reduit_indexes],
        "value": static_sample_individu_df.eurometropole_strasbourg_tarification_solidaire_transport_quotient_familial[no_reduit_indexes]
    })

    no_reduit_rr_res_desc = no_reduit_rr_res.groupby(by="rank").value.describe()
    no_reduit_rr_res_desc

    no_reduit_rr_res_desc["50%"].round().astype('int64')